### 연구 목적: Indoor moving patterns을 이용하여 revisit intention을 예측.
<pre>

</pre>
__어떤 유저의 Revisit intention(Revisit count)을 예측하는 supervised learning(regression) 모델에 이용될 간단한 Feature들:__
1. User가 찍힌 로그 총 개수: num_logs
2. 한 User가 와이파이에 잡힌 총 시간: total_dwell_time
3. dwell_time > 10인 indoor area 개수: num_sp_100
4. indoor 로그 중 dwell_time > 10인 확률: prob_dwell_10
5. 전체 로그 중 deny=True(직원)인 확률: prob_deny
6. dwell_time > 100인 indoor area에서 보낸 total time: time_sp_100
7. dwell_time > 100인 indoor area들의 variance: std_sp_100



__여기서의 초점__
1. 한 유저에 대한 log가 여러 개인데, 일단 aggregate하여 유저당 하나의 row로 나타낼 수 있도록 함)
2. 가지고 있는 데이터셋의 revisit_count는 같은 날 두 번 방문하면 올라가지 않는다 - 유저별 max(revisit_count) > 0 이면 yes, 아니면 no로 레이블링
3. 정확한 트레이닝은, 특정 시점 이전의 데이터를 이용하여 특정 시점 이후에 고객이 방문했는지 안 했는지 조사해야 하지만, 귀찮으므로 일단은 지금까지 전체 데이터를 가지고 revisit count를 예측하는 모델로 구성

__데이터__
1. 781, 786번 매장 데이터 (10%의 유저를 테스트로 이용)

In [4]:
### import libraries
import pandas as pd
import datetime
import numpy as np

In [5]:
df = pd.read_pickle("../data/781/781.p")

In [94]:
# len(df['device_id'].unique())
# uniqIDs = df['device_id'].unique()
# columns = ['num_logs', 'total_dwell_time', 'num_area_10', 'prob_dwell_10', 'prob_deny']

In [154]:
f1, f2, f3, f4 ,f5, f6, f7, label = feature_generator(df)

Generating features from raw data


In [155]:
df2 = df_generator(df, f1, f2, f3, f4, f5, f6, f7, label)

Generating a data frame which aggergated features


In [7]:
# Check methods works right by comparing with test dataframe
df2.loc[['0d4fd55bb363bf6f6f7f8b3342cd0467','15493306b1ea43056a690005e664d346','839d202915edc20699f08f733d46f030','f076ab559ef130c78ab7123d3807c1fc','f6258edf9145d1c0404e6f3d7a27a29d'],:]

NameError: name 'df2' is not defined

월화수목금토일 나누기  (1day = 86400 sec)
ts 기준
* 1472688000: 09/01/2016 @ 12:00am (UTC) Thursday
* 1472601600: 08/31/2016 @ 12:00am (UTC) Wednesday
* 86400 7         604800
* 나눠서 나머지가 0-86400이면 목요일, --- 순서대로 수요일

In [3]:
remainder = (df['ts']%604800000)/1000

In [130]:
def timestamp_to_day(x):
    a = x / 86400
    switcher = {
        0: "Thu",
        1: "Fri",
        2: "Sat",
        3: "Sun",
        4: "Mon",
        5: "Tue",
        6: "Wed"
    }
    return switcher.get(int(a))


df['day'] = remainder.apply(lambda x: timestamp_to_day(x))

### Test:  Small dataframe (2 people)

In [4]:
criterion = df['device_id'].map(lambda x: x.startswith('f6258edf9145d1c0404e6f3d7a27a29d') or x.startswith('0d4fd55bb363bf6f6f7f8b3342cd0467') or x.startswith('f076ab559ef130c78ab7123d3807c1fc') or x.startswith('839d202915edc20699f08f733d46f030') or x.startswith('15493306b1ea43056a690005e664d346'))
df_toy = df[criterion]

In [5]:
df_toy.shape

(25579, 8)

In [133]:
df_toy.columns

Index(['area', 'deny', 'device_id', 'dwell_time', 'key', 'revisit_count',
       'revisit_period', 'ts', 'day'],
      dtype='object')

In [134]:
df_toy['device_id'].unique()

array(['f076ab559ef130c78ab7123d3807c1fc',
       'f6258edf9145d1c0404e6f3d7a27a29d',
       '15493306b1ea43056a690005e664d346',
       '839d202915edc20699f08f733d46f030',
       '0d4fd55bb363bf6f6f7f8b3342cd0467'], dtype=object)

In [135]:
df.head()

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,day
0,1f-right-1,True,cae0f3cb170db4ae18897d6af8497c38,3330,781:7fea91f80d36bcc5:1f-right-1,NaN,NaN,1472645246100,Wed
1,right-test,True,cae0f3cb170db4ae18897d6af8497c38,3330,781:7fea91f80d36bcc5:right-test,NaN,NaN,1472645246100,Wed
2,out,NaN,c41e7932a5fedc55d61950559232c0bf,0,781:7fea91f80e6add4d:out,NaN,NaN,1472645241170,Wed
3,out,NaN,249e7ac1b1cd6ba9bee1cd81ef1e013f,85,781:7fea91f80ea49f2d:out,NaN,NaN,1472645240246,Wed
4,out,NaN,35e9e62a01594c31a9297c397cee0390,34,781:7fea91f80ee44f5d:out,NaN,NaN,1472645239227,Wed


In [169]:
# ### F8: 로그 총 개수 - 요일별

# # f8 = df_toy.groupby(['device_id', 'day'])['ts'].count()
# # print(f8)


# df_toy_indoor = df_toy.loc[df_toy['area']!='out']    ### F3이랑 중복
# df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]   ## F3이랑 중복
# f8 = df_toy_indoor2.groupby(['device_id', 'day'])['dwell_time'].sum()

# # df_toy_indoor2['day_of_week'] = df_toy_indoor2.apply(
# #     lambda x: ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'].index(x.day), 
# #     1
# # )
# # print(df_toy_indoor2.sort(['device_id', 'day_of_week']).groupby(['device_id', 'day'])['dwell_time'].sum())

# f8 = f8.reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], level='day')

# # print(type(df_toy_indoor2.groupby(['device_id', 'day'])['dwell_time']))
# print(f8, type(f8))

device_id                         day
0d4fd55bb363bf6f6f7f8b3342cd0467  Mon     826862
                                  Tue     792066
                                  Wed     636451
                                  Thu     436518
                                  Fri     770307
                                  Sat     953503
                                  Sun    1019298
f6258edf9145d1c0404e6f3d7a27a29d  Mon    2534895
                                  Tue    3303173
                                  Wed    3822616
                                  Thu    2857429
                                  Fri    3548178
                                  Sat    4475221
                                  Sun    4213633
Name: dwell_time, dtype: int64 <class 'pandas.core.series.Series'>


In [287]:
### F8: 로그 총 개수 - 요일별 (연습)

# f8 = df_toy.groupby(['device_id', 'day'])['ts'].count()
# print(f8)


df_toy_indoor = df_toy.loc[df_toy['area']!='out']    ### F3이랑 중복
df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]   ## F3이랑 중복
f8 = df_toy_indoor2.groupby(['day', 'device_id'])['dwell_time'].sum()

# df_toy_indoor2['day_of_week'] = df_toy_indoor2.apply(
#     lambda x: ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'].index(x.day), 
#     1
# )
# print(df_toy_indoor2.sort(['device_id', 'day_of_week']).groupby(['device_id', 'day'])['dwell_time'].sum())

f8 = f8.reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], level='day')

# print(type(df_toy_indoor2.groupby(['device_id', 'day'])['dwell_time']))
print(f8, type(f8))


day  device_id                       
Mon  0d4fd55bb363bf6f6f7f8b3342cd0467     826862
     f6258edf9145d1c0404e6f3d7a27a29d    2534895
Tue  0d4fd55bb363bf6f6f7f8b3342cd0467     792066
     f6258edf9145d1c0404e6f3d7a27a29d    3303173
Wed  0d4fd55bb363bf6f6f7f8b3342cd0467     636451
     f6258edf9145d1c0404e6f3d7a27a29d    3822616
Thu  0d4fd55bb363bf6f6f7f8b3342cd0467     436518
     f6258edf9145d1c0404e6f3d7a27a29d    2857429
Fri  0d4fd55bb363bf6f6f7f8b3342cd0467     770307
     f6258edf9145d1c0404e6f3d7a27a29d    3548178
Sat  0d4fd55bb363bf6f6f7f8b3342cd0467     953503
     f6258edf9145d1c0404e6f3d7a27a29d    4475221
Sun  0d4fd55bb363bf6f6f7f8b3342cd0467    1019298
     f6258edf9145d1c0404e6f3d7a27a29d    4213633
Name: dwell_time, dtype: int64 <class 'pandas.core.series.Series'>


In [288]:
f8 = f8.to_frame(name='count').reset_index()
f8

,day,device_id,count
0,Mon,0d4fd55bb363bf6f6f7f8b3342cd0467,826862
1,Mon,f6258edf9145d1c0404e6f3d7a27a29d,2534895
2,Tue,0d4fd55bb363bf6f6f7f8b3342cd0467,792066
3,Tue,f6258edf9145d1c0404e6f3d7a27a29d,3303173
4,Wed,0d4fd55bb363bf6f6f7f8b3342cd0467,636451
5,Wed,f6258edf9145d1c0404e6f3d7a27a29d,3822616
6,Thu,0d4fd55bb363bf6f6f7f8b3342cd0467,436518
7,Thu,f6258edf9145d1c0404e6f3d7a27a29d,2857429
8,Fri,0d4fd55bb363bf6f6f7f8b3342cd0467,770307
9,Fri,f6258edf9145d1c0404e6f3d7a27a29d,3548178


In [289]:
f8_2 = f8.loc[f8['day']=='Mon']
f8_3 = f8_2[["device_id", "count"]].set_index(['device_id'])
f8_3

,count
device_id,
0d4fd55bb363bf6f6f7f8b3342cd0467,826862
f6258edf9145d1c0404e6f3d7a27a29d,2534895


In [282]:
df_toy2

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_log_day,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun
0d4fd55bb363bf6f6f7f8b3342cd0467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15493306b1ea43056a690005e664d346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
839d202915edc20699f08f733d46f030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f076ab559ef130c78ab7123d3807c1fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f6258edf9145d1c0404e6f3d7a27a29d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
df_toy2['num_logs_Mon'] = f8_3

In [286]:
df_toy2

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_log_day,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun
0d4fd55bb363bf6f6f7f8b3342cd0467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,826862.0,NaN,NaN,NaN,NaN,NaN,NaN
15493306b1ea43056a690005e664d346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
839d202915edc20699f08f733d46f030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f076ab559ef130c78ab7123d3807c1fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f6258edf9145d1c0404e6f3d7a27a29d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2534895.0,NaN,NaN,NaN,NaN,NaN,NaN


In [291]:
### F8: 로그 총 개수 - 요일별 (연습)

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
df_toy_indoor = df_toy.loc[df_toy['area']!='out']    ### F3이랑 중복
df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]   ## F3이랑 중복
f8 = df_toy_indoor2.groupby(['day', 'device_id'])['dwell_time'].sum()
f8 = f8.reindex(days, level='day')
f8 = f8.to_frame(name='count').reset_index()


### F8를 df에 합치는 부분
for day in days:
    f8_certain_day = f8.loc[f8['day']==day]
    f8_certain_day = f8_certain_day[["device_id", "count"]].set_index(['device_id'])
    columnName = 'num_logs'+day
    df_toy2[columnName] = f8_certain_day



In [166]:
### F1: 로그 총 개수
# df_toy11 = df_toy['device_id'].value_counts()
f1 = df_toy.groupby(['device_id'])['ts'].count()
print(f1)

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467     3489
15493306b1ea43056a690005e664d346     3301
839d202915edc20699f08f733d46f030     2780
f076ab559ef130c78ab7123d3807c1fc     2375
f6258edf9145d1c0404e6f3d7a27a29d    13634
Name: ts, dtype: int64


In [80]:
### F2: 와이파이에 잡힌 총 시간
f2 = df_toy.groupby(['device_id'])['dwell_time'].sum()
f2

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467     8111450
15493306b1ea43056a690005e664d346      697601
839d202915edc20699f08f733d46f030     1068532
f076ab559ef130c78ab7123d3807c1fc     1054252
f6258edf9145d1c0404e6f3d7a27a29d    33565146
Name: dwell_time, dtype: int64

In [72]:
### F3: dwell_time > 100인 indoor area 개수
df_toy_indoor = df_toy.loc[df_toy['area']!='out']
df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]
f3 = df_toy_indoor2.groupby(['device_id'])['area'].count()
print(f3)

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    1725
f6258edf9145d1c0404e6f3d7a27a29d    7449
Name: area, dtype: int64


In [73]:
### F4: indoor 로그 중 dwell_time > 10인 확률
f3_2 = df_toy_indoor.groupby(['device_id'])['area'].count()
f4 = f3.div(f3_2)
print (f4)

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    0.542112
15493306b1ea43056a690005e664d346         NaN
839d202915edc20699f08f733d46f030         NaN
f6258edf9145d1c0404e6f3d7a27a29d    0.607140
Name: area, dtype: float64


In [74]:
### F5: deny = True일 확률
# df_toy_true = df_toy.loc[df_toy['deny']==True]
# df_toy.groupby(['device_id'])['deny'].count()
# print(df_toy.groupby(['device_id', 'deny']).count())
a = df_toy.groupby(['device_id']).deny.count()
b = df_toy['device_id'].value_counts()
f5 = a.div(b)
# print (a, b)
print (f5)

0d4fd55bb363bf6f6f7f8b3342cd0467    0.980797
15493306b1ea43056a690005e664d346    0.971305
839d202915edc20699f08f733d46f030    0.945026
f076ab559ef130c78ab7123d3807c1fc    0.925280
f6258edf9145d1c0404e6f3d7a27a29d    0.997272
dtype: float64


In [143]:
### F6: dwell_time > 100인 indoor area에서 보낸 total time

df_toy_indoor = df_toy.loc[df_toy['area']!='out']    ### F3이랑 중복
df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]   ## F3이랑 중복
f6 = df_toy_indoor2.groupby(['device_id'])['dwell_time'].sum()
print(f6)



device_id
0d4fd55bb363bf6f6f7f8b3342cd0467     5435005
f6258edf9145d1c0404e6f3d7a27a29d    24755145
Name: dwell_time, dtype: int64


In [145]:
### F7: dwell_time > 100인 indoor area들의 variance

df_toy_indoor = df_toy.loc[df_toy['area']!='out']    ### F3이랑 중복
df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]   ## F3이랑 중복
f7 = df_toy_indoor2.groupby(['device_id'])['dwell_time'].var()
print(f7)


device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    15107295
f6258edf9145d1c0404e6f3d7a27a29d    20541653
Name: dwell_time, dtype: int64


In [143]:
df_toy['tsafter'] = df_toy['ts']+(1000*df_toy['dwell_time'])

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [144]:
### F8: temporally annotated sequence

df_toy_839d = df_toy.loc[df_toy['device_id']=='f6258edf9145d1c0404e6f3d7a27a29d']


df_toy_839d['ts'] = df_toy_839d['ts'].apply(lambda x: datetime.datetime.fromtimestamp(int(x/1000)).strftime('%Y-%m-%d %H:%M:%S'))
df_toy_839d['tsafter'] = df_toy_839d['tsafter'].apply(lambda x: datetime.datetime.fromtimestamp(int(x/1000)).strftime('%Y-%m-%d %H:%M:%S'))

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [145]:
df_toy_839d.loc[df_toy_839d['dwell_time'] > 0][-800:-750]

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,tsafter
3649497,b1-left-2,True,f6258edf9145d1c0404e6f3d7a27a29d,554,781:7feb0070351d4867:b1-left-2,25.0,1.0,2015-09-23 15:55:53,2015-09-23 16:05:07
3649684,b1-right,True,f6258edf9145d1c0404e6f3d7a27a29d,3,781:7feb0070d9b2bf6a:b1-right,25.0,1.0,2015-09-23 15:44:39,2015-09-23 15:44:42
3649688,b1-center-1,True,f6258edf9145d1c0404e6f3d7a27a29d,1265,781:7feb0070e241d21b:b1-center-1,25.0,1.0,2015-09-23 15:44:04,2015-09-23 16:05:09
3649833,1f-right-1,True,f6258edf9145d1c0404e6f3d7a27a29d,395,781:7feb00716effbcf1:1f-right-1,25.0,1.0,2015-09-23 15:34:27,2015-09-23 15:41:02
3649834,right-test,True,f6258edf9145d1c0404e6f3d7a27a29d,395,781:7feb00716effbcf1:right-test,25.0,1.0,2015-09-23 15:34:27,2015-09-23 15:41:02
3649852,1f-right-2,True,f6258edf9145d1c0404e6f3d7a27a29d,413,781:7feb00717a608ada:1f-right-2,25.0,1.0,2015-09-23 15:33:41,2015-09-23 15:40:34
3651699,b1-center-2,True,f6258edf9145d1c0404e6f3d7a27a29d,425,781:7feb0078374fcad1:b1-center-2,25.0,1.0,2015-09-23 13:35:55,2015-09-23 13:43:00
3652666,b1-center-2,True,f6258edf9145d1c0404e6f3d7a27a29d,66,781:7feb007bc204f089:b1-center-2,25.0,1.0,2015-09-23 12:34:01,2015-09-23 12:35:07
3652718,b1-right,True,f6258edf9145d1c0404e6f3d7a27a29d,886,781:7feb007bf4280689:b1-right,25.0,1.0,2015-09-23 12:30:36,2015-09-23 12:45:22
3652913,b1-right,True,f6258edf9145d1c0404e6f3d7a27a29d,31,781:7feb007cadace4ef:b1-right,25.0,1.0,2015-09-23 12:17:56,2015-09-23 12:18:27


In [132]:
df_toy_839d.loc[df_toy_839d['area'].map(lambda x: x == 'in' or x == 'out')][-500:-470]

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts
2833999,in,True,f6258edf9145d1c0404e6f3d7a27a29d,12003,781:7feae7626b9e3fe2:in,79.0,1.0,2015-12-10 12:04:56
2834040,out,True,f6258edf9145d1c0404e6f3d7a27a29d,35084,781:7feae76289f5b2ae:out,79.0,1.0,2015-12-10 12:02:52
2838915,in,True,f6258edf9145d1c0404e6f3d7a27a29d,16202,781:7feae7a282793fa3:in,79.0,2.0,2015-12-09 17:24:54
2843417,in,True,f6258edf9145d1c0404e6f3d7a27a29d,18285,781:7feae7b4d1d45fe2:in,79.0,2.0,2015-12-09 12:04:54
2843450,out,True,f6258edf9145d1c0404e6f3d7a27a29d,35645,781:7feae7b4f6ac0225:out,79.0,2.0,2015-12-09 12:02:23
2854365,in,True,f6258edf9145d1c0404e6f3d7a27a29d,6505,781:7feae83dfb962c21:in,80.0,1.0,2015-12-07 20:07:48
2857453,in,True,f6258edf9145d1c0404e6f3d7a27a29d,11360,781:7feae84983f03e2c:in,80.0,1.0,2015-12-07 16:46:15
2857708,out,True,f6258edf9145d1c0404e6f3d7a27a29d,19871,781:7feae84abaecea5b:out,80.0,1.0,2015-12-07 16:25:02
2859343,in,True,f6258edf9145d1c0404e6f3d7a27a29d,5066,781:7feae8521a192f5a:in,80.0,1.0,2015-12-07 14:16:12
2859344,out,True,f6258edf9145d1c0404e6f3d7a27a29d,5186,781:7feae8521a20149b:out,80.0,1.0,2015-12-07 14:16:12


In [179]:
df['tsafter'] = df['ts']+(1000*df['dwell_time'])
df['ts'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(int(x/1000)).strftime('%Y-%m-%d %H:%M:%S'))
df['tsafter'] = df['tsafter'].apply(lambda x: datetime.datetime.fromtimestamp(int(x/1000)).strftime('%Y-%m-%d %H:%M:%S'))
df.loc[df['area'].map(lambda x: x == 'in' or x == 'out')]

In [185]:
df.loc[df['area'].map(lambda x: x == 'in' or x == 'out')]

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,tsafter
0,out,NaN,8fed73dd321afba08da3bd714424f53a,0,781:7fea94dad5796cc0:out,NaN,NaN,2016-08-22 21:56:16,2016-08-22 21:56:16
1,out,NaN,0f63d2e16a5b27e4e483dadda7f1784e,0,781:7fea94dad7979c5b:out,NaN,NaN,2016-08-22 21:56:08,2016-08-22 21:56:08
2,out,NaN,ad36531ac891adeac6e8783d7b60cf6f,0,781:7fea94dad92b8f29:out,NaN,NaN,2016-08-22 21:56:01,2016-08-22 21:56:01
3,out,NaN,b47a5604842db1c1845e27c75f248664,0,781:7fea94dad93b9f97:out,NaN,NaN,2016-08-22 21:56:01,2016-08-22 21:56:01
4,out,NaN,dad9ff8a20456a05a985a70bd763bd76,19,781:7fea94dad9baee76:out,1.0,35.0,2016-08-22 21:55:59,2016-08-22 21:56:18
5,out,NaN,995ed1ea617fc868f427550969b671bc,4,781:7fea94dad9bbfcf6:out,NaN,NaN,2016-08-22 21:55:59,2016-08-22 21:56:03
6,out,NaN,fc4213001d1df2ae214bea939f3deb1e,3,781:7fea94dad9edff63:out,NaN,NaN,2016-08-22 21:55:58,2016-08-22 21:56:01
7,out,NaN,00e1cc3ad6d33b6bf2db831a2769bb1e,2,781:7fea94dadbd1ad62:out,NaN,NaN,2016-08-22 21:55:50,2016-08-22 21:55:52
8,out,NaN,4045be9a18d26c0d9e02679001db46a7,8,781:7fea94dadc24bfa5:out,NaN,NaN,2016-08-22 21:55:49,2016-08-22 21:55:57
9,out,True,1cca2db136168337466c619c1ab802f1,0,781:7fea94dadcdb7e4d:out,NaN,NaN,2016-08-22 21:55:46,2016-08-22 21:55:46


시간을 정보를 모조리 뭉뚱그린 하나의 feature로 아래 그림과 같은 sequence를 나타내긴 아까운데...
![예제](20160830_223133_HDR.jpg)

### 방법이 없을까?
Temporally annotated sequence를 feature로 사용하는 방법?

1. [A Brief Survey on Sequence Classification](../documents/Sequence_Classification.pdf)
2. 

In [92]:
# df_toy2['device_id'] = 
np.sort(df_toy['device_id'].unique())

array(['0d4fd55bb363bf6f6f7f8b3342cd0467',
       '15493306b1ea43056a690005e664d346',
       '839d202915edc20699f08f733d46f030',
       'f076ab559ef130c78ab7123d3807c1fc',
       'f6258edf9145d1c0404e6f3d7a27a29d'], dtype=object)

In [279]:
device_ids_toy = np.sort(df_toy['device_id'].unique())

In [280]:
df_toy2 = pd.DataFrame(columns=columns, index=device_ids_toy)

In [37]:
# df_toy2['device_id'] = np.sort(df_toy['device_id'].unique())

In [281]:
df_toy2

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_log_day,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun
0d4fd55bb363bf6f6f7f8b3342cd0467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15493306b1ea43056a690005e664d346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
839d202915edc20699f08f733d46f030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f076ab559ef130c78ab7123d3807c1fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f6258edf9145d1c0404e6f3d7a27a29d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_toy2["num_logs"] = f1
df_toy2["total_dwell_time"] = f2
df_toy2["num_area_10"] = f3
df_toy2["prob_dwell_10"] = f4
df_toy2["prob_deny"] = f5

In [57]:
df_toy2

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny
0d4fd55bb363bf6f6f7f8b3342cd0467,3489,8111450,1725.0,0.542112,0.980797
15493306b1ea43056a690005e664d346,3241,689201,NaN,NaN,0.971305
839d202915edc20699f08f733d46f030,2674,1042562,NaN,NaN,0.945026
f076ab559ef130c78ab7123d3807c1fc,2235,974213,NaN,NaN,0.925280
f6258edf9145d1c0404e6f3d7a27a29d,13196,33024302,7449.0,0.607140,0.997272


In [61]:
##### 삽질한 코드들#####
# pd.concat([df_toy2, f4], axis=1)
# df_toy2.merge(f1, on='device_id', how='left')
# df_toy2['total_dwell_time'] = df_toy.groupby(['device_id'])['dwell_time'].sum()
# df_toy2 = pd.concat([df_toy2, df_toy12], axis=1)
# df_toy12f = df_toy12.to_frame()
# df_toy12.get

In [66]:
label_toy = df_toy.groupby(['device_id'])['revisit_count'].max()
label_toy

device_id
0d4fd55bb363bf6f6f7f8b3342cd0467    68.0
15493306b1ea43056a690005e664d346     4.0
839d202915edc20699f08f733d46f030     3.0
f076ab559ef130c78ab7123d3807c1fc     NaN
f6258edf9145d1c0404e6f3d7a27a29d    81.0
Name: revisit_count, dtype: float64

In [69]:
pd.concat([df_toy2, label_toy], axis=1)

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny,revisit_count
device_id,,,,,,
0d4fd55bb363bf6f6f7f8b3342cd0467,3489,8111450,1725.0,0.542112,0.980797,68.0
15493306b1ea43056a690005e664d346,3241,689201,NaN,NaN,0.971305,4.0
839d202915edc20699f08f733d46f030,2674,1042562,NaN,NaN,0.945026,3.0
f076ab559ef130c78ab7123d3807c1fc,2235,974213,NaN,NaN,0.925280,NaN
f6258edf9145d1c0404e6f3d7a27a29d,13196,33024302,7449.0,0.607140,0.997272,81.0


### 정리하기

In [155]:
criterion = df['device_id'].map(lambda x: x.startswith('f6258edf9145d1c0404e6f3d7a27a29d') or x.startswith('0d4fd55bb363bf6f6f7f8b3342cd0467') or x.startswith('f076ab559ef130c78ab7123d3807c1fc') or x.startswith('839d202915edc20699f08f733d46f030') or x.startswith('15493306b1ea43056a690005e664d346'))
df_toy = df[criterion]

In [4]:
def feature_generator(df_toy):
    ### F1: 로그 총 개수
    f1 = df_toy.groupby(['device_id'])['ts'].count()
#     print(f1)
    
    ### F2: 와이파이에 잡힌 총 시간
    f2 = df_toy.groupby(['device_id'])['dwell_time'].sum()
#     print(f2)

    ### F3: dwell_time > 100인 indoor area 개수
    df_toy_indoor = df_toy.loc[df_toy['area']!='out']
    df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]
    f3 = df_toy_indoor2.groupby(['device_id'])['area'].count()
#     print(f3)

    ### F4: indoor 로그 중 dwell_time > 10인 확률
    f3_2 = df_toy_indoor.groupby(['device_id'])['area'].count()
    f4 = f3.div(f3_2)
#     print (f4)
    
    ### F5: deny = True일 확률
    a = df_toy.groupby(['device_id']).deny.count()
    b = df_toy['device_id'].value_counts()
    f5 = a.div(b)
#     print (f5)

    ### Label: Maximum revisit count from the log
    label_toy = df_toy.groupby(['device_id'])['revisit_count'].max()
#     print(label_toy)

    return f1, f2, f3, f4, f5, label_toy

In [104]:
f1_toy, f2_toy, f3_toy, f4_toy ,f5_toy, label_toy = feature_generator(df_toy)

In [9]:
def df_generator(df, f1, f2, f3, f4, f5, label):
    columns = ['num_logs', 'total_dwell_time', 'num_area_10', 'prob_dwell_10', 'prob_deny']
    
    # feature들과의 index의 통일을 위해 np.sort를 이용.
    device_ids = np.sort(df['device_id'].unique())       
    df2 = pd.DataFrame(columns=columns, index=device_ids)
    
    # feature를 df에 삽입
    df2["num_logs"] = f1          
    df2["total_dwell_time"] = f2
    df2["num_area_10"] = f3
    df2["prob_dwell_10"] = f4
    df2["prob_deny"] = f5
    
    # label을 df에 합침
    df2 = pd.concat([df2, label], axis=1)   
    
    # machine learning에 바로 이용될 dataframe을 리턴
    return df2
    

In [107]:
df_toy2 = df_generator(df_toy, f1_toy, f2_toy, f3_toy, f4_toy, f5_toy, label_toy)

In [151]:
df_toy2

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny,revisit_count
device_id,,,,,,
0d4fd55bb363bf6f6f7f8b3342cd0467,3489,8111450,1725.0,0.542112,0.980797,68.0
15493306b1ea43056a690005e664d346,3241,689201,NaN,NaN,0.971305,4.0
839d202915edc20699f08f733d46f030,2674,1042562,NaN,NaN,0.945026,3.0
f076ab559ef130c78ab7123d3807c1fc,2235,974213,NaN,NaN,0.925280,NaN
f6258edf9145d1c0404e6f3d7a27a29d,13196,33024302,7449.0,0.607140,0.997272,81.0


### 데이터 정리

In [44]:
# df2.loc[df2.revisit_count > 0, 'revisit_count'] = 1
df2 = df2.fillna(0)
data = np.asarray(df2)
X, y = data[:, 0:-1], data[:, -1]
indices = np.random.permutation(len(X))
X_train = X[indices[:450000]]
y_train = y[indices[:450000]]
X_test = X[indices[450000:500000]]
y_test = y[indices[450000:500000]]

### 간단한 Linear Regression Model
   1. 데이터 프레임을 Array로 바꾼 후 진행
   2. Coefficient과 Intercept를 구함
   3. 더 할 부분: Train/test를 나누고 Loss를 계산하기 - 이전에 Logistic regression으로 해서 accuracy 측정을 해보자

In [45]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
LinearRegression(copy_X=True, fit_intercept=True, normalize=False)
print('Coefficient: ', lr.coef_)
print('Intercept: ', lr.intercept_)
print(np.mean((lr.predict(X_test) - y_test)**2))
print(lr.score(X_test, y_test))

Coefficient:  [  7.11805310e-04   2.61081644e-06   5.17519322e-02   2.04101452e-01
   4.10617292e-01   4.01522135e-04   1.73469890e+03  -1.03445124e-06
   2.44482415e-07  -2.20387583e-07   1.41743537e-08   1.01065378e-10
  -2.07247303e-10  -9.09494702e-13  -1.73469892e+03  -1.73469884e+03
  -1.73469861e+03  -1.73469895e+03  -1.73469891e+03  -1.73469898e+03
  -1.73469916e+03]
Intercept:  0.00976409687471
0.883481025228
-6.4278166196


### 간단한 Logistic Regression Model
   1. 데이터 프레임을 Array로 바꾼 후 진행
   2. Coefficient과 Intercept를 구함
   3. 더 할 부분: Train/test를 나누고 Loss를 계산하기 - 이전에 Logistic regression으로 해서 accuracy 측정을 해보자

### SGD Classifier

In [122]:
from sklearn.linear_model import SGDClassifier

In [124]:
clf = SGDClassifier(loss="hinge", penalty="l2")
clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [126]:
y_pred_sgd = clf.predict(X_test)

In [127]:
confusion_matrix(y_pred_sgd, y_test)

array([[1616,    0],
       [3309,   75]])

### KNN Classifier

In [41]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

y_pred = knn.predict(X_test)

By definition, entry i, j in a confusion matrix is the number of observations actually in group i, but predicted to be in group j

In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

array([[4916,   75],
       [   9,    0]])

In [49]:
np.bincount(y_test.astype(int))

array([4925,   75])

In [56]:
from sklearn.metrics import classification_report
target_names = ['No revisit', 'Revisit']
print(classification_report(y_test, y_pred, target_names = target_names))

             precision    recall  f1-score   support

 No revisit       0.98      1.00      0.99      4925
    Revisit       0.00      0.00      0.00        75

avg / total       0.97      0.98      0.98      5000



# XGBoost  (실제 작동 데모는 여기부터)

In [1]:
__author__ = 'ZFTurbo: https://kaggle.com/zfturbo'
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import random
import zipfile
import time
import shutil
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error

random.seed(2016)

In [30]:
datadir = "../data/781/781.p"

df = pd.read_pickle(datadir)

In [3]:
remainder = (df['ts']%604800000)/1000

def timestamp_to_day(x):
    a = x / 86400
    switcher = {
        0: "Thu",
        1: "Fri",
        2: "Sat",
        3: "Sun",
        4: "Mon",
        5: "Tue",
        6: "Wed"
    }
    return switcher.get(int(a))


df['day'] = remainder.apply(lambda x: timestamp_to_day(x))

In [4]:
def feature_generator(df_toy):
    print('Generating features from raw data')
    ### F1: 로그 총 개수
    f1 = df_toy.groupby(['device_id'])['ts'].count()

    ### F2: 와이파이에 잡힌 총 시간
    f2 = df_toy.groupby(['device_id'])['dwell_time'].sum()

    ### F3: dwell_time > 100인 indoor area 개수
    df_toy_indoor = df_toy.loc[df_toy['area']!='out']
    df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]
    f3 = df_toy_indoor2.groupby(['device_id'])['area'].count()

    ### F4: indoor 로그 중 dwell_time > 10인 확률
    f3_2 = df_toy_indoor.groupby(['device_id'])['area'].count()
    f4 = f3.div(f3_2)

    ### F5: deny = True일 확률
    a = df_toy.groupby(['device_id']).deny.count()
    b = df_toy['device_id'].value_counts()
    f5 = a.div(b)

    ### F6: dwell_time > 100인 indoor area에서 보낸 total time
    f6 = df_toy_indoor2.groupby(['device_id'])['dwell_time'].std()

    ### F7: dwell_time > 100인 indoor area들의 variance
    f7 = df_toy_indoor2.groupby(['device_id'])['dwell_time'].sum()
    
    ### F8: 로그 총 개수 - 요일별
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    f8 = df_toy_indoor2.groupby(['day', 'device_id'])['dwell_time'].sum()
    
    ### Label: Maximum revisit count from the log
    label_toy = df_toy.groupby(['device_id'])['revisit_count'].max()

    return f1, f2, f3, f4, f5, f6, f7, f8, label_toy

In [9]:
def df_generator(df, f1, f2, f3, f4, f5, f6, f7, f8, label):
    print('Generating a data frame which aggergated features')
    
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    days_numlogs = ['num_logs_' + s for s in days]
    columns = ['num_logs', 'total_dwell_time', 'num_sp_100', 'prob_dwell_10', 'prob_deny', 'time_sp_100', 'std_sp_100']
    columns = columns + days_numlogs

    # feature들과의 index의 통일을 위해 np.sort를 이용.
    device_ids = np.sort(df['device_id'].unique())       
    df2 = pd.DataFrame(columns=columns, index=device_ids)

    # feature를 df에 삽입
    df2["num_logs"] = f1          
    df2["total_dwell_time"] = f2
    df2["num_sp_100"] = f3
    df2["prob_dwell_10"] = f4
    df2["prob_deny"] = f5
    df2["time_sp_100"] = f6
    df2["std_sp_100"] = f7
    
    ### F8를 df에 합치는 부분
    for day in days:
        f8_certain_day = f8.loc[f8['day']==day]
        f8_certain_day = f8_certain_day[["device_id", "count"]].set_index(['device_id'])
        columnName = 'num_logs'+day
        df2[columnName] = f8_certain_day

    # label을 df에 합침
    df2 = pd.concat([df2, label], axis=1)   

    # machine learning에 바로 이용될 dataframe을 리턴
    return df2

In [6]:
f1, f2, f3, f4 ,f5, f6, f7, f8, label = feature_generator(df)

Generating features from raw data


In [10]:
df2 = df_generator(df, f1, f2, f3, f4, f5, f6, f7, f8, label)

Generating a data frame which aggergated features


In [11]:
df2 = df2.fillna(0)
df2 = df2.reindex(np.random.permutation(df2.index))

In [20]:
idx = int(len(df2.index)*9/10)
train = df2[:idx]
test = df2[idx:]
features = list(df2.columns)[:-1]
target = 'revisit_count'

In [38]:
def run_xgb(train, test, features, target, random_state=0):
    start_time = time.time()
    objective = "reg:linear"
    booster = "gbtree"
    eval_metric = ['rmse', 'auc']
    eta = 0.1
    max_depth = 3
    subsample = 0.7
    colsample_bytree = 0.7
    silent = 1

    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    params = {
        "objective": objective,
    #         "num_class": 2,
        "booster" : booster,
        "eval_metric": eval_metric,
        "eta": eta,
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": silent,
        "seed": random_state,
    }
    num_boost_round = 200
    early_stopping_rounds = 20
    test_size = 0.2

    X_train, X_valid = train_test_split(train, test_size=test_size, random_state=random_state)
    print('Length train:', len(X_train.index))
    print('Length valid:', len(X_valid.index))
    y_train = X_train[target]
    y_valid = X_valid[target]
    dtrain = xgb.DMatrix(X_train[features], y_train)
    dvalid = xgb.DMatrix(X_valid[features], y_valid)

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
    gbm.save_model('0001.model')
    gbm.dump_model('dump.raw.txt')
    xgb.plot_importance(gbm)
#     xgb.plot_tree(gbm, num_trees=2)
    
    print("Validating...")
    check = gbm.predict(xgb.DMatrix(X_valid[features]), ntree_limit=gbm.best_iteration)

    score = mean_squared_error(y_valid.tolist(), check)

    print("Predict test set...")
    test_prediction = gbm.predict(xgb.DMatrix(test[features]), ntree_limit=gbm.best_iteration)

    training_time = round((time.time() - start_time)/60, 2)
    print('Training time: {} minutes'.format(training_time))

    print(gbm)

    # To save logs
    explog = {}
    explog['features'] = features
    explog['target'] = target
    explog['params'] = {}
    explog['params']['objective'] = objective
    explog['params']['booster'] = booster
    explog['params']['eval_metric'] = eval_metric
    explog['params']['eta'] = eta
    explog['params']['max_depth'] = max_depth
    explog['params']['subsample'] = subsample
    explog['params']['colsample_bytree'] = colsample_bytree
    explog['params']['silent'] = silent
    explog['params']['seed'] = random_state
    explog['params']['num_boost_round'] = num_boost_round
    explog['params']['early_stopping_rounds'] = early_stopping_rounds
    explog['params']['test_size'] = test_size
    explog['length_train']= len(X_train.index)
    explog['length_valid']= len(X_valid.index)
    # explog['gbm_best_iteration']= 
    explog['score'] = score
    explog['training_time'] = training_time




    return test_prediction.tolist(), score, explog

In [22]:
print('Length of train: ', len(train))
print('Length of test: ', len(test))
print('Features [{}]: {}'.format(len(features), sorted(features)))

Length of train:  1406270
Length of test:  156253
Features [21]: ['num_logs', 'num_logsFri', 'num_logsMon', 'num_logsSat', 'num_logsSun', 'num_logsThu', 'num_logsTue', 'num_logsWed', 'num_logs_Fri', 'num_logs_Mon', 'num_logs_Sat', 'num_logs_Sun', 'num_logs_Thu', 'num_logs_Tue', 'num_logs_Wed', 'num_sp_100', 'prob_deny', 'prob_dwell_10', 'std_sp_100', 'time_sp_100', 'total_dwell_time']


In [39]:
test_prediction, score, explog = run_xgb(train, test, features, target)

XGBoost params. ETA: 0.1, MAX_DEPTH: 3, SUBSAMPLE: 0.7, COLSAMPLE_BY_TREE: 0.7
Length train: 1125016
Length valid: 281254


Will train until eval error hasn't decreased in 20 rounds.
Multiple eval metrics have been passed: 'auc' will be used for early stopping.

[0]	train-rmse:0.524830	train-auc:0.869744	eval-rmse:0.521057	eval-auc:0.870458
[1]	train-rmse:0.478381	train-auc:0.869778	eval-rmse:0.478498	eval-auc:0.870465
[2]	train-rmse:0.439185	train-auc:0.869802	eval-rmse:0.443118	eval-auc:0.870588
[3]	train-rmse:0.401321	train-auc:0.871904	eval-rmse:0.408112	eval-auc:0.872737
[4]	train-rmse:0.368748	train-auc:0.961372	eval-rmse:0.376553	eval-auc:0.961856
[5]	train-rmse:0.338800	train-auc:0.962233	eval-rmse:0.347406	eval-auc:0.962535
[6]	train-rmse:0.313074	train-auc:0.960236	eval-rmse:0.324802	eval-auc:0.961100
[7]	train-rmse:0.290365	train-auc:0.960234	eval-rmse:0.301676	eval-auc:0.961103
[8]	train-rmse:0.269346	train-auc:0.960234	eval-rmse:0.283194	eval-auc:0.961102
[9]	train-rmse:0.251431	train-auc:0.960341	eval-rmse:0.266552	eval-auc:0.961227
[10]	train-rmse:0.236349	train-auc:0.964430	eval-rmse:0.25373

Validating...
Predict test set...
Training time: 0.35 minutes


In [42]:
import json

In [40]:
def updateLog(explog, logPath):
    try:
        with open(logPath, 'r') as f:
            obob = json.load(f)
        f.close()
    except:
        obob = []


    obob.append(explog)

    with open(logPath, 'w') as f:
        json.dump(obob, f)
    f.close()

In [43]:
logPath = '../result/results.json'

explog['dataset']= datadir
explog['ts']= time.strftime('%Y-%m-%d %H:%M:%S')

updateLog(explog, logPath)

In [25]:
print(explog)

{'training_time': 1.76, 'score': 0.020556798473463655, 'features': ['num_logs', 'total_dwell_time', 'num_sp_100', 'prob_dwell_10', 'prob_deny', 'time_sp_100', 'std_sp_100', 'num_logs_Mon', 'num_logs_Tue', 'num_logs_Wed', 'num_logs_Thu', 'num_logs_Fri', 'num_logs_Sat', 'num_logs_Sun', 'num_logsMon', 'num_logsTue', 'num_logsWed', 'num_logsThu', 'num_logsFri', 'num_logsSat', 'num_logsSun'], 'params': {'max_depth': 3, 'booster': 'gbtree', 'eval_metric': 'rmse', 'objective': 'reg:linear', 'eta': 0.1}, 'current_time': 1472754875.180503, 'target': 'revisit_count'}


In [65]:
import json

outputPath = '../result/results.json'
try:
    with open(outputPath, 'r') as f:
        obob = json.load(f)
    f.close()
except:
    obob = []

    
obob.append(explog)

with open(outputPath, 'w') as f:
    json.dump(obob, f)
f.close()

In [73]:
with open(outputPath, 'r') as f:
    obob = json.load(f)
    f.close()



In [74]:
obob

[{'current_time': 1472716002.204983,
  'features': ['num_logs',
   'total_dwell_time',
   'num_sp_100',
   'prob_dwell_10',
   'prob_deny',
   'time_sp_100',
   'std_sp_100'],
  'params': {'booster': 'gbtree',
   'eta': 0.1,
   'eval_metric': 'rmse',
   'max_depth': 3,
   'objective': 'reg:linear'},
  'score': 0.023240665525514598,
  'target': 'revisit_count',
  'training_time': 0.23},
 {'dataset': '../data/781/781.p',
  'features': ['num_logs',
   'total_dwell_time',
   'num_sp_100',
   'prob_dwell_10',
   'prob_deny',
   'time_sp_100',
   'std_sp_100'],
  'length_train': 984389,
  'length_valid': 421881,
  'params': {'booster': 'gbtree',
   'colsample_bytree': 0.7,
   'early_stopping_rounds': 1,
   'eta': 0.1,
   'eval_metric': 'rmse',
   'max_depth': 3,
   'num_boost_round': 0.7,
   'objective': 'reg:linear',
   'seed': 0,
   'silent': 1,
   'subsample': 0.7,
   'test_size': 0},
  'score': 0.025487321227262133,
  'target': 'revisit_count',
  'training_time': 0.29,
  'ts': '2016-09-0

In [24]:
resulttxt

NameError: name 'resulttxt' is not defined

In [39]:
def updateJsonFile():
 jsonFile = open("replayScript.json", "r")
 data = json.load(jsonFile)
 jsonFile.close()


 tmp = data["location"]
 data["location"] = path
 data["mode"] = "replay"

 jsonFile = open("replayScript.json", "w+")
 jsonFile.write(json.dumps(data))
 jsonFile.close()

{'features': ['num_logs', 'total_dwell_time', 'num_sp_100', 'prob_dwell_10', 'prob_deny', 'time_sp_100', 'std_sp_100'], 'target': 'revisit_count', 'score': 0.01903936318706075, 'params': {'booster': 'gbtree', 'eta': 0.1, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'max_depth': 3}, 'training_time': 0.82}


In [171]:
tp = pd.DataFrame(test_prediction)

In [172]:
type(test['revisit_count'])

pandas.core.series.Series

In [173]:
compare = pd.concat([tp, test.reset_index()['revisit_count']], axis=1, ignore_index=True)

compare = compare.rename(columns={0: 'pred', 1: 'true'})

compare.sort(['true'], ascending=False)

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,pred,true
71233,36.679951,44.0
70994,43.034863,41.0
142789,12.765020,20.0
33085,1.333457,11.0
23122,0.780250,9.0
145205,15.631070,9.0
76017,5.127707,9.0
2478,6.000453,7.0
121690,1.465179,7.0
111189,1.409659,7.0


In [174]:
score

0.022564888741160829

In [175]:
train['revisit_count'].value_counts().sort_index()

0.0     1357014
1.0       14693
2.0        1239
3.0         259
4.0          91
5.0          47
6.0          19
7.0          17
8.0          16
9.0           9
10.0          5
11.0          5
12.0          2
13.0          7
14.0          1
15.0          3
16.0          3
18.0          1
19.0          2
20.0          2
22.0          1
23.0          1
29.0          1
31.0          1
33.0          2
34.0          1
37.0          1
38.0          1
39.0          1
51.0          1
52.0          1
54.0          1
56.0          3
63.0          2
68.0          1
70.0          1
72.0          2
78.0          2
81.0          1
90.0          5
Name: revisit_count, dtype: int64

In [73]:
tp[(tp[0]>0.25)].count()

0    469
dtype: int64

In [74]:
tp.sort([0],  ascending=False )

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,0
12423,13.468298
17765,8.876219
45146,2.643633
47700,2.077081
49347,1.939507
45582,1.821067
10382,1.735981
1710,1.721989
9618,1.436606
31537,1.396812


In [214]:
tp.sort([1],  ascending=False )[270:280]

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,0,1
41356,0.667452,0.332548
3588,0.668275,0.331725
2566,0.668356,0.331644
1939,0.668379,0.331621
21556,0.669054,0.330946
37879,0.669543,0.330457
49756,0.669773,0.330227
25142,0.670018,0.329982
5240,0.670029,0.329971
17485,0.670177,0.329824


In [217]:
test[1939:1940]

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny,revisit_count
device_id,,,,,,
4be7ae5b564467d2da8df5a23f5f3783,10,5932,3.0,0.375,0.0,0.0


In [75]:
score

0.022347515020357291

### 검증해보려 하는데 뭔가 이상해 ... 실제 데이터랑 인덱스가 안맞아

In [73]:
np.where(y_pred==1)

(array([ 682, 1699, 1795, 2218, 2898, 3317, 3839, 4238, 4425]),)

In [72]:
np.where(y_test==1)

(array([ 136,  151,  254,  267,  316,  376,  398,  458,  461,  504,  732,
         776,  777,  788,  815,  828,  847,  892,  913,  972, 1018, 1086,
        1107, 1148, 1233, 1401, 1418, 1432, 1526, 1691, 1826, 1883, 1886,
        2074, 2090, 2273, 2366, 2410, 2505, 2571, 2636, 2665, 2941, 3129,
        3172, 3219, 3329, 3452, 3465, 3626, 3667, 3705, 3731, 3806, 3812,
        3909, 3967, 3992, 3999, 4063, 4115, 4147, 4409, 4411, 4448, 4484,
        4615, 4668, 4673, 4691, 4768, 4805, 4934, 4995, 4997]),)

In [93]:
device_ids = np.sort(df['device_id'].unique())

In [115]:
df2[45000:50000].loc[df2['revisit_count']==1].index[0:3]

Index(['0793a3c28ba18a69819605906793c725', '079ca4d8168ed6853d4763cb87598db7',
       '079cae1cc6a1e83777404af258a64a51'],
      dtype='object', name='device_id')

In [116]:
df2.loc[df2.index == '0793a3c28ba18a69819605906793c725']

,num_logs,total_dwell_time,num_area_10,prob_dwell_10,prob_deny,revisit_count
device_id,,,,,,
0793a3c28ba18a69819605906793c725,14,287,0.0,0.0,0.0,1.0


In [120]:
np.where(device_ids =='0793a3c28ba18a69819605906793c725')

(array([45040]),)

## 교수님이 원하는 방향 (각 방문을 index로 구현)

In [6]:
df_toy.shape

(25579, 8)

In [7]:
df_toy.head(10)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts
418,out,True,f076ab559ef130c78ab7123d3807c1fc,595,781:7fea91f9744caffd:out,NaN,NaN,1472643775285
547,1f-right-2,True,f6258edf9145d1c0404e6f3d7a27a29d,0,781:7fea91f9ea5f1fd4:1f-right-2,NaN,NaN,1472643291662
548,1f-right-1,True,f6258edf9145d1c0404e6f3d7a27a29d,0,781:7fea91f9ea66dd9a:1f-right-1,NaN,NaN,1472643291538
562,1f,True,f6258edf9145d1c0404e6f3d7a27a29d,63,781:7fea91f9f9e28f1a:1f,NaN,NaN,1472643228119
563,in,True,f6258edf9145d1c0404e6f3d7a27a29d,63,781:7fea91f9f9e28f1a:in,NaN,NaN,1472643228119
564,out,True,f6258edf9145d1c0404e6f3d7a27a29d,63,781:7fea91f9f9eb6fd0:out,NaN,NaN,1472643227977
580,out,True,15493306b1ea43056a690005e664d346,0,781:7fea91fa0c30ace2:out,NaN,NaN,1472643153141
606,out,True,839d202915edc20699f08f733d46f030,0,781:7fea91fa2abd9ddb:out,NaN,NaN,1472643028006
817,1f-right-1,True,f6258edf9145d1c0404e6f3d7a27a29d,0,781:7fea91faef3f0e2c:1f-right-1,NaN,NaN,1472642223119
818,right-test,True,f6258edf9145d1c0404e6f3d7a27a29d,0,781:7fea91faef3f0e2c:right-test,NaN,NaN,1472642223119


In [8]:
df_toy['date'] = df_toy['ts'] // 86400000

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
df_toy.head(5)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date
418,out,True,f076ab559ef130c78ab7123d3807c1fc,595,781:7fea91f9744caffd:out,NaN,NaN,1472643775285,17044
547,1f-right-2,True,f6258edf9145d1c0404e6f3d7a27a29d,0,781:7fea91f9ea5f1fd4:1f-right-2,NaN,NaN,1472643291662,17044
548,1f-right-1,True,f6258edf9145d1c0404e6f3d7a27a29d,0,781:7fea91f9ea66dd9a:1f-right-1,NaN,NaN,1472643291538,17044
562,1f,True,f6258edf9145d1c0404e6f3d7a27a29d,63,781:7fea91f9f9e28f1a:1f,NaN,NaN,1472643228119,17044
563,in,True,f6258edf9145d1c0404e6f3d7a27a29d,63,781:7fea91f9f9e28f1a:in,NaN,NaN,1472643228119,17044


In [10]:
dfdict = dict.fromkeys(df_toy.date.unique())

In [11]:
df_toy = df_toy.loc[(df_toy['deny']!= 'True') & df_toy['dwell_time'] > 0]

In [14]:
for day in dfdict.keys():
    uniqids = df_toy.loc[df_toy['date'] == day]['device_id'].unique()
    dfdict[day] = dict.fromkeys(uniqids)    
    for ide in dfdict[day].keys():
        dfdict[day][ide] = df_toy.loc[(df_toy['date']==day) & (df_toy['device_id'] == ide)]

In [16]:
len(dfdict)

370

### 뭔가 망한듯 ... 다시

In [1]:
### import libraries
import pandas as pd
import datetime
import numpy as np

In [6]:
df = pd.read_pickle("../data/781/781.p")
df['date'] = df['ts'] // 86400000
# df = df.loc[(df['deny']!= True) & (df['dwell_time'] > 0)]
df = df.loc[(df['dwell_time'] > 0)]
df['date_device_id'] = df.date.map(str) + "_" + df.device_id

In [7]:
remainder = (df['ts']%604800000)/1000

def timestamp_to_day(x):
    a = x / 86400
    switcher = {
        0: "Thu",
        1: "Fri",
        2: "Sat",
        3: "Sun",
        4: "Mon",
        5: "Tue",
        6: "Wed"
    }
    return switcher.get(int(a))

df['day'] = remainder.apply(lambda x: timestamp_to_day(x))

In [8]:
def feature_generator(df_toy):
    print('Generating features from raw data')
    ### F1: 로그 총 개수
    f1 = df_toy.groupby(['date_device_id'])['ts'].count()
#     print(f1.head(5))

    ### F2: 와이파이에 잡힌 총 시간
    f2 = df_toy.groupby(['date_device_id'])['dwell_time'].sum()
#     print(f2.head(5))

    ### F3: dwell_time > 100인 indoor area 개수
    df_toy_indoor = df_toy.loc[df_toy['area']!='out']
    df_toy_indoor2 = df_toy_indoor.loc[df_toy_indoor['dwell_time']>100]
    f3 = df_toy_indoor2.groupby(['date_device_id'])['area'].count()
#     print(f3.head(5))

    ### F4: indoor 로그 중 dwell_time > 100인 확률
    f3_2 = df_toy_indoor.groupby(['date_device_id'])['area'].count()
    f4 = f3.div(f3_2)
#     print(f4.head(5))

    ### F5: deny = True일 확률
    a = df_toy.groupby(['date_device_id']).deny.count()
    b = df_toy['date_device_id'].value_counts()
    f5 = a.div(b)
#     print(f5.head(5))

    ### F6: dwell_time > 100인 indoor area에서 보낸 total time
    f6 = df_toy_indoor2.groupby(['date_device_id'])['dwell_time'].sum()
#     print(f6.head(5))

    ### F7: dwell_time > 100인 indoor area들의 standard deviation
    f7 = df_toy_indoor2.groupby(['date_device_id'])['dwell_time'].std()
#     print(f7.head(5))
    
    ### F8: 로그 총 개수 - 요일별
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    f8 = df_toy_indoor2.groupby(['day', 'date_device_id'])['dwell_time'].sum()
    f8 = f8.reindex(days, level='day')
    f8 = f8.to_frame(name='count').reset_index()
#     print(f8.head(5))
    
    ### Label: Maximum revisit count from the log
    label_toy = df_toy.groupby(['date_device_id'])['revisit_count'].max()
#     print(label_toy.head(5))

    return f1, f2, f3, f4, f5, f6, f7, f8, label_toy

In [136]:
df.head(3)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
0,1f-right-1,True,cae0f3cb170db4ae18897d6af8497c38,3330,781:7fea91f80d36bcc5:1f-right-1,NaN,NaN,1472645246100,17044,17044_cae0f3cb170db4ae18897d6af8497c38,Wed
1,right-test,True,cae0f3cb170db4ae18897d6af8497c38,3330,781:7fea91f80d36bcc5:right-test,NaN,NaN,1472645246100,17044,17044_cae0f3cb170db4ae18897d6af8497c38,Wed
3,out,NaN,249e7ac1b1cd6ba9bee1cd81ef1e013f,85,781:7fea91f80ea49f2d:out,NaN,NaN,1472645240246,17044,17044_249e7ac1b1cd6ba9bee1cd81ef1e013f,Wed


In [5]:
f1, f2, f3, f4, f5, f6, f7, f8, label = feature_generator(df)

Generating features from raw data


In [6]:
d2 = df.groupby(['date_device_id'])['dwell_time'].sum()
d2 = d2.reindex(df.date_device_id.unique().tolist())
d2 = d2.to_frame(name='sum').reset_index()

In [139]:
f8.shape

(10866, 3)

In [7]:
def df_generator(df, f1, f2, f3, f4, f5, f6, f7, f8, label):
    print('Generating a data frame which aggergated features')

    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    days_numlogs = ['num_logs_' + s for s in days]
    columns = ['num_logs', 'total_dwell_time', 'num_sp_100', 'prob_dwell_10', 'prob_deny', 'time_sp_100', 'std_sp_100']
    columns = columns + days_numlogs

    # feature들과의 index의 통일을 위해 np.sort를 이용.
    device_ids = np.sort(df['date_device_id'].unique())       
    df2 = pd.DataFrame(columns=columns, index=device_ids)

    # feature를 df에 삽입
    df2["num_logs"] = f1          
    df2["total_dwell_time"] = f2
    df2["num_sp_100"] = f3
    df2["prob_dwell_10"] = f4
    df2["prob_deny"] = f5
    df2["time_sp_100"] = f6
    df2["std_sp_100"] = f7

    ### F8를 df에 합치는 부분
    for day in days:
        f8_certain_day = f8.loc[f8['day']==day]
        f8_certain_day = f8_certain_day[["date_device_id", "count"]].set_index(['date_device_id'])
        columnName = 'num_logs_'+day
        df2[columnName] = f8_certain_day

    # label을 df에 합침
    df2 = pd.concat([df2, label], axis=1)   

    # machine learning에 바로 이용될 dataframe을 리턴
    return df2

In [8]:
df2 = df_generator(df, f1, f2, f3, f4, f5, f6, f7, f8, label)

Generating a data frame which aggergated features


In [9]:
df2.head(5)

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun,revisit_count
date_device_id,,,,,,,,,,,,,,,
16673_000563414d92ae0e06674c1d8d5e24a4,1,3069,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_000c27de1f619c696368fa7ed2df92c1,1,345,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00242425ce6b95736d3161a000569c2d,1,35,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_0037261f7aa87d2d5f31d1ecad98cc26,1,361,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00708c2793e18784096dc9a5d65eec1e,1,177,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df.loc[df['date_device_id']=='17044_fc62d0d2e721590d6c008bd91112e8b4']

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
2368,out,True,fc62d0d2e721590d6c008bd91112e8b4,377,781:7fea91ff9018ef68:out,1.0,37.0,1472637369969,17044,17044_fc62d0d2e721590d6c008bd91112e8b4,Wed
4666,out,True,fc62d0d2e721590d6c008bd91112e8b4,15,781:7fea92075ba62e8c:out,1.0,37.0,1472629196189,17044,17044_fc62d0d2e721590d6c008bd91112e8b4,Wed
7800,out,True,fc62d0d2e721590d6c008bd91112e8b4,1155,781:7fea9214fcc77cca:out,1.0,37.0,1472614904712,17044,17044_fc62d0d2e721590d6c008bd91112e8b4,Wed


In [ ]:
d6 = df.loc[df['dwell_time']>=100].groupby(['date_device_id'])['dwell_time'].sum()
d6 = d6.reindex(df.date_device_id.unique().tolist())
d6 = d6.to_frame(name='total_dwell_time').reset_index()

In [119]:
# df[['day', 'date_device_id']].groupby(['date_device_id']).sum().shape

In [104]:
### 아직 안 쓴 쓴 features
df[['date_device_id', 'day']].set_index('date_device_id').shape

(2199729, 1)

In [121]:
df2.shape

(1831147, 15)

In [125]:
df2.sort_values('total_dwell_time', ascending=False).head(10)

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun,revisit_count
date_device_id,,,,,,,,,,,,,,,
16842_6c3483f617ad92e18cd36a7ed680fcad,2,123235,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
16785_b6a2c7c0fea10bda8a9664a235a3eb85,2,87074,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
16896_f6258edf9145d1c0404e6f3d7a27a29d,28,75042,24.0,0.923077,0.0,52108.0,3579.923143,NaN,52108.0,NaN,NaN,NaN,NaN,NaN,NaN
16834_6c3483f617ad92e18cd36a7ed680fcad,7,72684,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
16835_6c3483f617ad92e18cd36a7ed680fcad,13,68272,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
16846_ff1ad58f042d41162c7328440ab6253c,1,66801,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16682_d16c36ff62b04a89a51e68bd0b9c7a82,14,48015,11.0,0.846154,0.0,9314.0,294.432706,NaN,NaN,NaN,NaN,9314.0,NaN,NaN,7.0
16842_ec517dd2142eebd789e6f61c33a911e9,7,47432,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16680_c96d2af5dde757bf8caa8712818d984a,5,46990,4.0,1.000000,0.0,2100.0,0.000000,NaN,NaN,2100.0,NaN,NaN,NaN,NaN,6.0


In [128]:
df2.loc[df2.num_logs == 1].sort_values('revisit_count', ascending=False).head(10)

# num_logs = 1(out만 있는) 경우에도 revisit_count가 자연수인 경우가 있다. 

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun,revisit_count
date_device_id,,,,,,,,,,,,,,,
16907_903e4ad38234df1b1f84464bf6b28dd5,1,513,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0
16906_903e4ad38234df1b1f84464bf6b28dd5,1,440,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0
16868_903e4ad38234df1b1f84464bf6b28dd5,1,2687,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0
16941_4bc4c56129373828b7a5784c95cd4519,1,796,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0
16945_4bc4c56129373828b7a5784c95cd4519,1,936,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0
16939_4bc4c56129373828b7a5784c95cd4519,1,12807,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
16974_4bc4c56129373828b7a5784c95cd4519,1,150,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0
16874_903e4ad38234df1b1f84464bf6b28dd5,1,3806,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0
16872_903e4ad38234df1b1f84464bf6b28dd5,1,160,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0


In [149]:
df.loc[df.device_id=='0d4fd55bb363bf6f6f7f8b3342cd0467'].sort_values('ts', ascending=False).head(5)

### df 데이터프레임은 직원을 없앤 데이터 결과물(deny=True를 제거), 아래 결과물은 device_id = '0d4fd55bb363bf6f6f7f8b3342cd0467'
### 에 해당되는 고객 정보인데, revisit_count 순서가 엉망이다 (max 60까지 갔었는데 다시 59, 7로 떨어짐 - 두 명이 같은 id로 겹친 느낌, 그리고 초반부를 보면 29부터 시작이다 - deny=true인 부분이랑 합쳐서 다시 봐야할듯.)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
1443522,b1,True,0d4fd55bb363bf6f6f7f8b3342cd0467,9987,781:7feabb3fd6743655:b1,57.0,1.0,1461564119228,16916,16916_0d4fd55bb363bf6f6f7f8b3342cd0467,Mon
1443523,in,True,0d4fd55bb363bf6f6f7f8b3342cd0467,9987,781:7feabb3fd6743655:in,57.0,1.0,1461564119228,16916,16916_0d4fd55bb363bf6f6f7f8b3342cd0467,Mon
1443527,out,True,0d4fd55bb363bf6f6f7f8b3342cd0467,10170,781:7feabb3fdbd15c25:out,57.0,1.0,1461564097258,16916,16916_0d4fd55bb363bf6f6f7f8b3342cd0467,Mon
1446363,b1,True,0d4fd55bb363bf6f6f7f8b3342cd0467,1474,781:7feabb4b7985134f:b1,57.0,1.0,1461551916974,16916,16916_0d4fd55bb363bf6f6f7f8b3342cd0467,Mon
1446364,in,True,0d4fd55bb363bf6f6f7f8b3342cd0467,1474,781:7feabb4b7985134f:in,57.0,1.0,1461551916974,16916,16916_0d4fd55bb363bf6f6f7f8b3342cd0467,Mon


In [158]:
df.loc[df.device_id=='903e4ad38234df1b1f84464bf6b28dd5'].sort_values('ts', ascending=False).head(3)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
629563,out,NaN,903e4ad38234df1b1f84464bf6b28dd5,9079,781:7feaa371b84bbf6e:out,5.0,12.0,1467954264900,16990,16990_903e4ad38234df1b1f84464bf6b28dd5,Fri
631182,out,NaN,903e4ad38234df1b1f84464bf6b28dd5,744,781:7feaa377f164fcf2:out,5.0,12.0,1467947739568,16990,16990_903e4ad38234df1b1f84464bf6b28dd5,Fri
632701,out,NaN,903e4ad38234df1b1f84464bf6b28dd5,3140,781:7feaa3acbcbacd67:out,5.0,11.0,1467892380755,16989,16989_903e4ad38234df1b1f84464bf6b28dd5,Thu


In [12]:
df.loc[df['device_id']=='e2efc41139f4a71f8aee77c35e5b7cbf'].sort_values('revisit_period', ascending=False)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
154367,out,NaN,e2efc41139f4a71f8aee77c35e5b7cbf,7,781:7fea96257ec34d44:out,1.0,90.0,1471523853259,17031,17031_e2efc41139f4a71f8aee77c35e5b7cbf,Thu
162333,out,NaN,e2efc41139f4a71f8aee77c35e5b7cbf,546,781:7fea963d18114d1f:out,1.0,90.0,1471499108075,17031,17031_e2efc41139f4a71f8aee77c35e5b7cbf,Thu
506825,out,NaN,e2efc41139f4a71f8aee77c35e5b7cbf,1036,781:7feaa02113744d09:out,1.0,59.0,1468844132539,17000,17000_e2efc41139f4a71f8aee77c35e5b7cbf,Mon
508100,out,NaN,e2efc41139f4a71f8aee77c35e5b7cbf,499,781:7feaa02507581d8e:out,1.0,59.0,1468839987838,17000,17000_e2efc41139f4a71f8aee77c35e5b7cbf,Mon
1169877,out,NaN,e2efc41139f4a71f8aee77c35e5b7cbf,176,781:7feab32fba775e46:out,NaN,NaN,1463728494730,16941,16941_e2efc41139f4a71f8aee77c35e5b7cbf,Fri


In [20]:
df.loc[df['device_id']=='bc4573cb1e7583be35ef91cc2522de01'].sort_values('revisit_count', ascending=False)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
2939255,1f-right-1,True,bc4573cb1e7583be35ef91cc2522de01,353,781:7feae79f0c6d6235:1f-right-1,90.0,1.0,1449653123369,16778,16778_bc4573cb1e7583be35ef91cc2522de01,Wed
2686841,1f-right-2,True,bc4573cb1e7583be35ef91cc2522de01,4667,781:7feae0830601f2d9:1f-right-2,90.0,1.0,1451561557984,16800,16800_bc4573cb1e7583be35ef91cc2522de01,Thu
2807465,1f-right-1,True,bc4573cb1e7583be35ef91cc2522de01,1343,781:7feae3bf897ffbd6:1f-right-1,90.0,1.0,1450692798464,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2813141,1f-right-1,True,bc4573cb1e7583be35ef91cc2522de01,420,781:7feae3d406293300:1f-right-1,90.0,1.0,1450671316332,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2814179,1f-right-1,True,bc4573cb1e7583be35ef91cc2522de01,3882,781:7feae3d8a83bfe66:1f-right-1,90.0,1.0,1450666458176,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2814217,1f,True,bc4573cb1e7583be35ef91cc2522de01,36306,781:7feae3d8ce00ebb9:1f,90.0,1.0,1450666303473,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2814218,1f-right-2,True,bc4573cb1e7583be35ef91cc2522de01,36306,781:7feae3d8ce00ebb9:1f-right-2,90.0,1.0,1450666303473,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2814219,in,True,bc4573cb1e7583be35ef91cc2522de01,36306,781:7feae3d8ce00ebb9:in,90.0,1.0,1450666303473,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2814221,out,True,bc4573cb1e7583be35ef91cc2522de01,36307,781:7feae3d8ce2871ee:out,90.0,1.0,1450666302840,16790,16790_bc4573cb1e7583be35ef91cc2522de01,Mon
2817045,right-test,True,bc4573cb1e7583be35ef91cc2522de01,19,781:7feae41122a6d244:right-test,90.0,1.0,1450607236498,16789,16789_bc4573cb1e7583be35ef91cc2522de01,Sun


In [24]:
# df['date_device_id'].value_counts()
# df.loc[df['date_device_id'] == '16898_cae0f3cb170db4ae18897d6af8497c38']
print ('a')


a


In [32]:
df.loc[df['date_device_id']=='17044_ff27ffc3ae6a9f6c7e2784fe6cea017f']

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
456,out,True,ff27ffc3ae6a9f6c7e2784fe6cea017f,789,781:7fea91f9931aecd8:out,NaN,NaN,1472643649105,17044,17044_ff27ffc3ae6a9f6c7e2784fe6cea017f,Wed
3068,out,True,ff27ffc3ae6a9f6c7e2784fe6cea017f,215,781:7fea9201b864ae4b:out,NaN,NaN,1472635107765,17044,17044_ff27ffc3ae6a9f6c7e2784fe6cea017f,Wed
3836,out,True,ff27ffc3ae6a9f6c7e2784fe6cea017f,1808,781:7fea9204296eef6c:out,NaN,NaN,1472632547601,17044,17044_ff27ffc3ae6a9f6c7e2784fe6cea017f,Wed
5581,out,True,ff27ffc3ae6a9f6c7e2784fe6cea017f,2316,781:7fea920b40b7fc4a:out,NaN,NaN,1472625112192,17044,17044_ff27ffc3ae6a9f6c7e2784fe6cea017f,Wed
6594,out,True,ff27ffc3ae6a9f6c7e2784fe6cea017f,202,781:7fea920fab455e08:out,NaN,NaN,1472620481450,17044,17044_ff27ffc3ae6a9f6c7e2784fe6cea017f,Wed
6828,out,True,ff27ffc3ae6a9f6c7e2784fe6cea017f,72,781:7fea9210e01dfe51:out,NaN,NaN,1472619216416,17044,17044_ff27ffc3ae6a9f6c7e2784fe6cea017f,Wed


In [9]:
dfsmall = df.loc[df['device_id']=='ceb0005b0d1e75950118f92c16d9a619']

In [10]:
trajsmall = dfsmall.groupby(['date', 'device_id'])['area']

In [11]:
trajsmallsum = trajsmall.sum()

In [12]:
# df.groupby(['date', 'device_id'])['area'].sum()
traj = df.groupby(['date', 'device_id'])['area']
# .map(lambda x: checkin(x))

In [13]:
# trajsmall.apply(lambda x: ','.join(x)[::])
# trajsmall.apply(lambda x: x)
trajformovings = traj.apply(lambda x: ','.join(x.sort_index(ascending=False))[::])

KeyboardInterrupt: 

In [173]:
# trajsmall.apply(lambda x: x.sort_index(ascending=False))

In [176]:
# trajsmall.apply(lambda x: ','.join(x.sort_index(ascending=False))[::])

In [70]:
trajsmallsum

date   device_id                       
16673  028a1f4dbca00ed06814fdda60f1b599    b1-left-1b1-center-1b1-rightb11f-left-11f-left...
16744  028a1f4dbca00ed06814fdda60f1b599                                     b1-left-3b1inout
Name: area, dtype: object

In [181]:
traj1 = trajformovings.to_frame(name='traj').reset_index()

In [182]:
df2.shape

(1876809, 15)

In [44]:
trajsum = traj.sum()

In [38]:
def checkin(x):
    result = 1
    if "inout" in x: 
        result = 1
    else:
        result = 0
    return result

In [49]:
df.shape

(2381255, 11)

In [143]:
trajcount = trajsum.map(lambda x: checkin(x)).groupby(level=1).cumsum()
traj2 = trajcount.to_frame(name='new_visit_count').reset_index()

In [184]:
traj1['date_device_id'] = traj1.date.map(str) + "_" + traj1.device_id
traj2['date_device_id'] = traj2.date.map(str) + "_" + traj2.device_id


In [185]:
traj1 = traj1.set_index(['date_device_id'])
traj2 = traj2.set_index(['date_device_id'])

In [186]:
trajs = pd.concat([traj1, traj2[['new_visit_count']]], axis=1)

In [201]:
trajs.loc[trajs['new_visit_count']==3].tail(5)

,date,device_id,traj,new_visit_count
date_device_id,,,,
17043_73519de7668af01b9567def349479daf,17043,73519de7668af01b9567def349479daf,out,3
17043_dad9ff8a20456a05a985a70bd763bd76,17043,dad9ff8a20456a05a985a70bd763bd76,out,3
17044_15179b79b52c49c97c1fe8e92c839312,17044,15179b79b52c49c97c1fe8e92c839312,out,3
17044_52cc32c37b65ce40861fab07faae3831,17044,52cc32c37b65ce40861fab07faae3831,"out,out",3
17044_73519de7668af01b9567def349479daf,17044,73519de7668af01b9567def349479daf,out,3


trajs를 각 device_id로 묶은 다음에(groupby) new_visit_count 값으로 정렬. 인덱스가 i인 row의 맨 첫값과 , i+1인 맨 첫값의 차이가 6months 이하일 경우 revisit_intention = 1

In [309]:
rajssmall.tail(3)

,date,device_id,traj,new_visit_count
date_device_id,,,,
17043_73519de7668af01b9567def349479daf,17043,73519de7668af01b9567def349479daf,out,3
17044_52cc32c37b65ce40861fab07faae3831,17044,52cc32c37b65ce40861fab07faae3831,"out,out",3
17044_73519de7668af01b9567def349479daf,17044,73519de7668af01b9567def349479daf,out,3


In [315]:
rajssmall['revisit_intention'] = 0

In [312]:
### 가능성 확인
rajssmall.set_value('16946_73519de7668af01b9567def349479daf', 'revisit_intention', 1)

,date,device_id,traj,new_visit_count,revisit_intention
date_device_id,,,,,
16792_73519de7668af01b9567def349479daf,16792,73519de7668af01b9567def349479daf,"out,in,1f,left-test,1f-left-2,b1",1,0
16800_73519de7668af01b9567def349479daf,16800,73519de7668af01b9567def349479daf,out,1,0
16804_73519de7668af01b9567def349479daf,16804,73519de7668af01b9567def349479daf,"out,out",1,0
16828_73519de7668af01b9567def349479daf,16828,73519de7668af01b9567def349479daf,out,1,0
16849_73519de7668af01b9567def349479daf,16849,73519de7668af01b9567def349479daf,out,1,0
16878_73519de7668af01b9567def349479daf,16878,73519de7668af01b9567def349479daf,out,1,0
16888_73519de7668af01b9567def349479daf,16888,73519de7668af01b9567def349479daf,out,1,0
16914_52cc32c37b65ce40861fab07faae3831,16914,52cc32c37b65ce40861fab07faae3831,"out,out,in,b1,b1-right,b1-center-1",1,0
16917_73519de7668af01b9567def349479daf,16917,73519de7668af01b9567def349479daf,out,1,0


In [14]:
trajssmall

NameError: name 'trajssmall' is not defined

In [317]:
rajssmall = trajssmall.loc[trajssmall.new_visit_count>=1]
rajssmall['revisit_intention'] = 0
revisit_interval_thres = 120

for ids in trajssmall['device_id'].unique():
    dff = trajssmall.loc[trajssmall['device_id']==ids]   
    a = 0
    date = 16672
    prev_idx = ''
    for index, row in dff.iterrows():
        if a+1 == row['new_visit_count']:
            if date+revisit_interval_thres > row['date']:
                print('regular revisit: {0} days interval'.format(row['date']-date))
                print('previous index: ',prev_idx)
                rajssmall.set_value(prev_idx, 'revisit_intention', 1)
                ## 이때만 하기.
                
            elif row['new_visit_count'] == 1:
                print('regular revisit: {0} days interval'.format(row['date']-date))
                
            else:
                print('Irregular revisit: {0} days interval'.format(row['date']-date))
            
            
            

            prev_idx = index
                
            print(row,'\n')
            a = row['new_visit_count']
            date = row['date']
            
            
        

regular revisit: 120 days interval
date                                          16792
device_id          73519de7668af01b9567def349479daf
traj               out,in,1f,left-test,1f-left-2,b1
new_visit_count                                   1
Name: 16792_73519de7668af01b9567def349479daf, dtype: object 

Irregular revisit: 154 days interval
date                                          16946
device_id          73519de7668af01b9567def349479daf
traj                                      out,in,b1
new_visit_count                                   2
Name: 16946_73519de7668af01b9567def349479daf, dtype: object 

regular revisit: 47 days interval
previous index:  16946_73519de7668af01b9567def349479daf
date                                          16993
device_id          73519de7668af01b9567def349479daf
traj                                  out,out,in,b1
new_visit_count                                   3
Name: 16993_73519de7668af01b9567def349479daf, dtype: object 

regular revisit: 242 days in

In [ ]:
rajssmall

In [248]:
# trajssmall.groupby(['device_id'])['date', 'new_visit_count'].apply(lambda x: x)  #이건 아님
trajssmall.sort(['device_id', 'date'], ascending=[1, 1]).apply(lambda x: )

/Users/sundong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,date,device_id,traj,new_visit_count
date_device_id,,,,
16680_52cc32c37b65ce40861fab07faae3831,16680,52cc32c37b65ce40861fab07faae3831,out,0
16696_52cc32c37b65ce40861fab07faae3831,16696,52cc32c37b65ce40861fab07faae3831,out,0
16854_52cc32c37b65ce40861fab07faae3831,16854,52cc32c37b65ce40861fab07faae3831,out,0
16861_52cc32c37b65ce40861fab07faae3831,16861,52cc32c37b65ce40861fab07faae3831,out,0
16874_52cc32c37b65ce40861fab07faae3831,16874,52cc32c37b65ce40861fab07faae3831,out,0
16883_52cc32c37b65ce40861fab07faae3831,16883,52cc32c37b65ce40861fab07faae3831,out,0
16891_52cc32c37b65ce40861fab07faae3831,16891,52cc32c37b65ce40861fab07faae3831,out,0
16895_52cc32c37b65ce40861fab07faae3831,16895,52cc32c37b65ce40861fab07faae3831,out,0
16910_52cc32c37b65ce40861fab07faae3831,16910,52cc32c37b65ce40861fab07faae3831,out,0


In [280]:
trajssmall

,date,device_id,traj,new_visit_count
date_device_id,,,,
16792_73519de7668af01b9567def349479daf,16792,73519de7668af01b9567def349479daf,"out,in,1f,left-test,1f-left-2,b1",1
16800_73519de7668af01b9567def349479daf,16800,73519de7668af01b9567def349479daf,out,1
16804_73519de7668af01b9567def349479daf,16804,73519de7668af01b9567def349479daf,"out,out",1
16828_73519de7668af01b9567def349479daf,16828,73519de7668af01b9567def349479daf,out,1
16849_73519de7668af01b9567def349479daf,16849,73519de7668af01b9567def349479daf,out,1
16878_73519de7668af01b9567def349479daf,16878,73519de7668af01b9567def349479daf,out,1
16888_73519de7668af01b9567def349479daf,16888,73519de7668af01b9567def349479daf,out,1
16914_52cc32c37b65ce40861fab07faae3831,16914,52cc32c37b65ce40861fab07faae3831,"out,out,in,b1,b1-right,b1-center-1",1
16917_73519de7668af01b9567def349479daf,16917,73519de7668af01b9567def349479daf,out,1


In [233]:
trajssmall = trajs.loc[(trajs['device_id'] == '52cc32c37b65ce40861fab07faae3831') | (trajs['device_id'] == '73519de7668af01b9567def349479daf')]

In [193]:
df2.loc[df2['total_dwell_time']> 10000]

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun,revisit_count
date_device_id,,,,,,,,,,,,,,,
16673_028a1f4dbca00ed06814fdda60f1b599,14,11092,11.0,0.846154,0.0,8329.0,870.565772,NaN,NaN,8329.0,NaN,NaN,NaN,NaN,NaN
16673_0516f6267b45fb4f22137defb83d5df4,6,20420,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_06d59ce48b0cedc7d60b485dcc1c8a90,3,20532,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_0d4fd55bb363bf6f6f7f8b3342cd0467,19,76002,16.0,0.888889,1.0,53715.0,4072.029735,NaN,NaN,53715.0,NaN,NaN,NaN,NaN,NaN
16673_1220be32db93db0ed3399cd7f7938941,4,16657,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_1f40ec338917cbeb661121057484bda9,1,15353,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_20569d57d629b9f046b69712b3611d30,5,13228,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_373b818105152ed3ea1e503a2690ad8e,5,27745,4.0,1.000000,0.0,1774.0,31.754265,NaN,NaN,1774.0,NaN,NaN,NaN,NaN,NaN
16673_387e29b6328bbc3e6ebbece99379f9dd,1,22299,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
df.loc[df['date_device_id']=='17044_ceb0005b0d1e75950118f92c16d9a619']

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,day
844,b1,True,ceb0005b0d1e75950118f92c16d9a619,4914,781:7fea91fb05c4bfd5:b1,NaN,NaN,1472642130868,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
845,in,True,ceb0005b0d1e75950118f92c16d9a619,4914,781:7fea91fb05c4bfd5:in,NaN,NaN,1472642130868,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
4104,b1,True,ceb0005b0d1e75950118f92c16d9a619,9747,781:7fea920509ec2d96:b1,NaN,NaN,1472631628093,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
4105,in,True,ceb0005b0d1e75950118f92c16d9a619,9747,781:7fea920509ec2d96:in,NaN,NaN,1472631628093,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
6467,b1,True,ceb0005b0d1e75950118f92c16d9a619,9632,781:7fea920f0e8b2e37:b1,NaN,NaN,1472621123405,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
6468,in,True,ceb0005b0d1e75950118f92c16d9a619,9632,781:7fea920f0e8b2e37:in,NaN,NaN,1472621123405,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
6806,b1,True,ceb0005b0d1e75950118f92c16d9a619,809,781:7fea9210c52a3f00:b1,NaN,NaN,1472619326812,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
6807,in,True,ceb0005b0d1e75950118f92c16d9a619,809,781:7fea9210c52a3f00:in,NaN,NaN,1472619326812,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
7736,b1,True,ceb0005b0d1e75950118f92c16d9a619,3059,781:7fea9214cdc60cfd:b1,NaN,NaN,1472615097247,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed
7737,in,True,ceb0005b0d1e75950118f92c16d9a619,3059,781:7fea9214cdc60cfd:in,NaN,NaN,1472615097247,17044,17044_ceb0005b0d1e75950118f92c16d9a619,Wed


In [62]:
traj.apply[traj['date_device_id']=='028a1f4dbca00ed06814fdda60f1b599']

Exception: Column(s) area already selected

In [64]:
trajsum2 = trajsum.to_frame(name='traj').reset_index()
# level='traj')
# trajsum3 = trajsum2.to_frame(name='traj')
# f8 = f8.to_frame(name='count').reset_index()
# f8 = f8.to_frame(name='count').reset_index()

In [65]:
trajsum2

,date,device_id,traj
0,16673,000563414d92ae0e06674c1d8d5e24a4,out
1,16673,000c27de1f619c696368fa7ed2df92c1,out
2,16673,00242425ce6b95736d3161a000569c2d,out
3,16673,0037261f7aa87d2d5f31d1ecad98cc26,out
4,16673,00708c2793e18784096dc9a5d65eec1e,out
5,16673,00762edef53140c9e18cfd2fc6c5aef6,out
6,16673,0085317900fbca13e9071b9208e77987,out
7,16673,008dfae6153c6a0049ce391ba9e969b2,out
8,16673,00962f3feed2d1b19d29cb265366d674,out
9,16673,00a82e129e43e4c9aa15274004beab46,out


In [56]:
df2.

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun,revisit_count
date_device_id,,,,,,,,,,,,,,,
16673_000563414d92ae0e06674c1d8d5e24a4,1,3069,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_000c27de1f619c696368fa7ed2df92c1,1,345,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00242425ce6b95736d3161a000569c2d,1,35,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_0037261f7aa87d2d5f31d1ecad98cc26,1,361,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00708c2793e18784096dc9a5d65eec1e,1,177,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00762edef53140c9e18cfd2fc6c5aef6,1,8,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_0085317900fbca13e9071b9208e77987,1,13,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_008dfae6153c6a0049ce391ba9e969b2,1,212,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00962f3feed2d1b19d29cb265366d674,1,10,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1

In [7]:
d9 = df.groupby(['date', 'device_id'])['dwell_time']

In [8]:
d9.apply(f1)

date   device_id                       
16673  000563414d92ae0e06674c1d8d5e24a4    3069
       000c27de1f619c696368fa7ed2df92c1     345
       00242425ce6b95736d3161a000569c2d      35
       0037261f7aa87d2d5f31d1ecad98cc26     361
       00708c2793e18784096dc9a5d65eec1e     177
       0085317900fbca13e9071b9208e77987      13
       00a82e129e43e4c9aa15274004beab46      33
       01125b7d994bd697cd67e163d1c89d12     277
       014b46fd526346713cde7525e8aa790b       7
       01592c162266e79e931996f4f8abfbcc       3
       0188fbc22a7a30f3c8051edc0facc068    1524
       01a9c238672f7dba51c3b22771b093bd     463
       01b9efdc342305e63d1c49ae6713ebc5    1051
       01c46eb30a7750f51b1c6efdb7f01263    3342
       01de3f0dfccafb3da4313c00e90ec236      15
       01e6f5686139006ffc779e24df102a48     827
       0235ca7b03a1264cecc4989b7411ba80      13
       023cfcb7f2fab3fc19a60527648ca3c4     267
       027d8258a4973b800300df0ebe9fc2f0      13
       028a1f4dbca00ed06814fdda60f1b599    9368


In [3]:
### Count stay points
def f1(x):
    #print (x)
    #print (x/x.max())
    #print (x/x.min())
    return sum(x)
#     return pd.Series(np.where(x>0, x/x.max(), x/x.min()), index=x.index)


# df1['normd'] = df1.groupby('id')['eff'].apply(f)

In [21]:
df2

,num_logs,total_dwell_time,num_sp_100,prob_dwell_10,prob_deny,time_sp_100,std_sp_100,num_logs_Mon,num_logs_Tue,num_logs_Wed,num_logs_Thu,num_logs_Fri,num_logs_Sat,num_logs_Sun,revisit_count
date_device_id,,,,,,,,,,,,,,,
16673_000563414d92ae0e06674c1d8d5e24a4,1,3069,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_000c27de1f619c696368fa7ed2df92c1,1,345,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00242425ce6b95736d3161a000569c2d,1,35,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_0037261f7aa87d2d5f31d1ecad98cc26,1,361,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00708c2793e18784096dc9a5d65eec1e,1,177,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00762edef53140c9e18cfd2fc6c5aef6,1,8,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_0085317900fbca13e9071b9208e77987,1,13,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_008dfae6153c6a0049ce391ba9e969b2,1,212,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16673_00962f3feed2d1b19d29cb265366d674,1,10,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df.loc[(df['date']==17044) & (df['device_id'] == '3392c7ac5ed5e33fef88c9dc753c1b58')]

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date
5870,1f-left-2,NaN,3392c7ac5ed5e33fef88c9dc753c1b58,9,781:7fea920c90fc4c49:1f-left-2,NaN,NaN,1472623734843,17044
5871,left-test,NaN,3392c7ac5ed5e33fef88c9dc753c1b58,9,781:7fea920c90fc4c49:left-test,NaN,NaN,1472623734843,17044
5908,1f-left-1,NaN,3392c7ac5ed5e33fef88c9dc753c1b58,209,781:7fea920cb1fdce7d:1f-left-1,NaN,NaN,1472623599651,17044
5940,1f-right-2,NaN,3392c7ac5ed5e33fef88c9dc753c1b58,327,781:7fea920cc9cd6cc7:1f-right-2,NaN,NaN,1472623502121,17044
5942,1f-right-1,NaN,3392c7ac5ed5e33fef88c9dc753c1b58,325,781:7fea920cca03cef8:1f-right-1,NaN,NaN,1472623501251,17044
5961,out,NaN,3392c7ac5ed5e33fef88c9dc753c1b58,433,781:7fea920cd4e92f5a:out,NaN,NaN,1472623456621,17044


In [69]:
df.loc[(df['date']==17044)]['device_id'].value_counts()

3392c7ac5ed5e33fef88c9dc753c1b58    6
30e53e1d1e1bf5e6b48e69af5cdab16a    6
20d662d5c47e4c5ddde18f5614977149    5
3cc761bc6cce9c78fb6afb421df63e66    5
8f8e13e048d1e9e27c3882757446d6f6    4
e21a280a75948e13c5e0ff853ec57dee    4
cc35eb326b0ad5bc686abb7c38ad7842    4
d07d4d5b668d2a12f4493be313c69378    4
320b91cea8e155f06be47d764682e635    3
a736b5fd7d4c5ac736584cd18b7ea48d    3
ae83da1a4a5cb027cf120b4088238e7b    3
e92714632d63703b9634cc646842dc75    3
21bb8ef49faf95b9d09d651234a7bf19    3
26d0c0b5bfc804d297e0beb27b9087ba    3
f5ee289013186ce23e6fc640af5670dc    3
f8fdb505d94e5c40f203e7fd2c374cf7    3
fa3fbe6f45ca4a8d96f39a1645a06e92    3
6df2d93168ff26d9ad599d3e9b16172d    3
a8310a8db32d1ea168933da346490f3b    3
166ed594e0c3fd2669310be242667ad3    3
edd613eff1ef40eb15b9199d7952cda9    3
3c6a10cbcce4979be3b8469a6f270741    3
46470519af01b5fec33c364be9715cb8    3
859a07ba0c55930138df5e1d4dac235b    3
06d59ce48b0cedc7d60b485dcc1c8a90    3
f380c1fec6d00dcb6b095faa99a9e830    3
b698201f0c63

In [9]:
df_toy.loc[df_toy['area']=='out']

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts
418,out,True,f076ab559ef130c78ab7123d3807c1fc,595,781:7fea91f9744caffd:out,NaN,NaN,2016-08-31 20:42:55
564,out,True,f6258edf9145d1c0404e6f3d7a27a29d,63,781:7fea91f9f9eb6fd0:out,NaN,NaN,2016-08-31 20:33:47
580,out,True,15493306b1ea43056a690005e664d346,0,781:7fea91fa0c30ace2:out,NaN,NaN,2016-08-31 20:32:33
606,out,True,839d202915edc20699f08f733d46f030,0,781:7fea91fa2abd9ddb:out,NaN,NaN,2016-08-31 20:30:28
944,out,True,839d202915edc20699f08f733d46f030,238,781:7fea91fb5c755caa:out,NaN,NaN,2016-08-31 20:09:35
1028,out,True,f076ab559ef130c78ab7123d3807c1fc,1332,781:7fea91fba0f86ffd:out,NaN,NaN,2016-08-31 20:04:55
1148,out,True,f6258edf9145d1c0404e6f3d7a27a29d,1061,781:7fea91fbf277ad53:out,NaN,NaN,2016-08-31 19:59:21
1391,out,True,839d202915edc20699f08f733d46f030,0,781:7fea91fcb1757e59:out,NaN,NaN,2016-08-31 19:46:19
1520,out,True,f076ab559ef130c78ab7123d3807c1fc,3,781:7fea91fd1b177d0a:out,NaN,NaN,2016-08-31 19:39:06
2017,out,True,839d202915edc20699f08f733d46f030,288,781:7fea91fe8236ac92:out,NaN,NaN,2016-08-31 19:14:35


In [72]:
time.strftime('%Y-%m-%d %H:%M:%S')

'2016-09-01 17:14:26'